<img src="../pics/python-snake.jpg" width="100%">


# Unix Domain Sockets vs TCP

Kevin J. Walchko, Phd

---

This is inspired by [Myhro Blog](https://blog.myhro.info/2017/01/how-fast-are-unix-domain-sockets)

In [78]:
import multiprocessing as mp
import socket
import time
import os

# Server and Client

Communicating via TCP or UDS is very similar, so here are some functions that create a server and a client. Each takes arguments to determine if communications is via TCP or UDS.

- Their address family: `socket.AF_INET` (IP) and `socket.AF_UNIX` (Unix sockets).
- To bind a process using `socket.AF_UNIX`, the socket file should be removed and created again if it already exists.
- When using `socket.AF_INET`, the `socket.SO_REUSEADDR` flag have to be set in order to avoid `socket.error: [Errno 98] Address already in use` errors that may occur even when the socket is properly closed. This option tells the kernel to reuse the same port if there are connections in the `TIME_WAIT` state.

## TCP

- allows communications between processes running on different machines
- all messages have to pass through the TCP/IP stack, so this should be slower

## UDS

- since the communications is through a file handler, the processes have to be on the same machine
    - I wouldn't use a file associated with a network share, then you are doing some sort of tcp/uds thing

In [68]:
def server(e,**kwargs):
    AF = kwargs['af']
    addr = kwargs['addr']
    
    duration = 5
    end = time.time() + duration

    sock = socket.socket(AF, socket.SOCK_STREAM)
    sock.setsockopt(socket.SOL_SOCKET, socket.SO_REUSEADDR, 1)
    sock.bind(addr)
    sock.listen(0)
    
    print("Server ready")

    conn, _ = sock.accept()
    
    while (time.time() < end) and e.is_set():
        conn.send(b'Hello there!')

    conn.close()
    print('*** Server is done ***')
    e.clear()

In [69]:
def client(e,**kwargs):
    AF = kwargs['af']
    addr = kwargs['addr']
    msgs = 0

    print('Receiving messages...')

    sock = socket.socket(AF, socket.SOCK_STREAM)
    sock.connect(addr)
    
    while e.is_set():
        data = sock.recv(32)
        msgs += 1
        
    sock.close()

    print('Received {} messages.'.format(msgs))

# TCP

In [89]:
event = mp.Event()
event.set()

host = socket.gethostbyname(socket.gethostname())
print("host computer: {}".format(host))

kw = {
    'af': socket.AF_INET,
    'addr': (host, 5000,)
}

s = mp.Process(target=server, args=(event,), name="server", kwargs=kw)
s.start()
print('Started {}[{}]'.format(s.name, s.pid))

host computer: 192.168.86.213
Started server[37995]
Server ready
*** Server is done ***


In [90]:
c = mp.Process(target=client, args=(event,), name="client", kwargs=kw)
c.start()
print('Started {}[{}]'.format(c.name, c.pid))

Started client[37996]
Receiving messages...
Received 64492 messages.


In [91]:
event.clear()

for p in [s, c]:
    p.join(0.1)
    if p.is_alive():
        p.terminate()
    print('{} is alive: {}'.format(p.name, p.is_alive()))

server is alive: False
client is alive: False


# UDS

In [95]:
event.set()

kw = {
    'af': socket.AF_UNIX,
    'addr': '/tmp/uds_test'
}

# if the file already exists, you will get an address in use error
if os.path.exists(kw['addr']):
    os.remove(kw['addr'])

s = mp.Process(target=server, args=(event,), name="server", kwargs=kw)
s.start()
print('Started {}[{}]'.format(s.name, s.pid))

Started server[37999]
Server ready
*** Server is done ***


In [96]:
c = mp.Process(target=client, args=(event,), name="client", kwargs=kw)
c.start()
print('Started {}[{}]'.format(c.name, c.pid))

Started client[38000]
Receiving messages...
Received 95507 messages.


In [97]:
event.clear()

for p in [s, c]:
    p.join(0.1)
    if p.is_alive():
        p.terminate()
    print('{} is alive: {}'.format(p.name, p.is_alive()))

server is alive: False
client is alive: False


# Results

These tests seem to be all over the place ... not sure if jupyter is interfering with the execution or not.

The original author listed above showed UDS consistantly being twice as fast as TCP.